## Set up environment

In [37]:
import json
import openai
# Read the configuration file
with open('config.json') as f:
    config = json.load(f)
# Get the API key from the configuration
api_key = config['OPENAI_API_KEY']
# Set up the OpenAI API client
openai.api_key = api_key

In [38]:
from langchain_openai.llms import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", api_key=api_key, max_tokens=10)
model.invoke("The sky is")

" falling.\n\nNo, I'm gonna be like,"

In [39]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(api_key=api_key)
prompt = [HumanMessage("What is the capital of France?")]

model.invoke(prompt)


AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEErmZBxbW0cOt3JrLbj33Y9opyFA', 'finish_reason': 'stop', 'logprobs': None}, id='run-52fe45fa-93ce-44c7-ab41-f7bf32b1fe35-0', usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [40]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatOpenAI(api_key=api_key)
system_msg = SystemMessage('''You are a helpful assistant that responds to questions with three exclamation marks.''')

human_msg = HumanMessage("What is the capital of France?")

model.invoke([system_msg, human_msg])


AIMessage(content='The capital of France is Paris!!!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 33, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEErnb6SsJbAx85RDFfQ0xgRQ2j3u', 'finish_reason': 'stop', 'logprobs': None}, id='run-8765c42a-39d1-4e88-a5cf-aa87051a6140-0', usage_metadata={'input_tokens': 33, 'output_tokens': 8, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Making LLM Prompts Reusable

In [ ]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Answer the question based on the
    context below. If the question cannot be answered using the information 
    provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer: """)

prompt = template.invoke({
    "context":"""The most recent advancements in NLP are being driven by Large Language 
Models (LLMs). These models outperform their smaller counterparts and have
become invaluable for developers who are creating applications with NLP 
capabilities. Developers can tap into these models through Hugging Face's
`transformers` library, or by utilizing OpenAI and Cohere's offerings through
the `openai` and `cohere` libraries, respectively.""",
    "question":"Which model providers offer LLMs?"
})

model = OpenAI(api_key=api_key)

completion = model.invoke(prompt)

In [36]:
print(completion)

Hugging Face, OpenAI, and Cohere all offer LLMs through their respective libraries.


In [41]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ('system', '''Answer the question based on the
    context below. If the question cannot be answered using the information 
    provided, answer with "I don't know".'''),
    ('human','Context: {context}'),
    ('human','Question: {question}'),
])


## Generate Specific Formats

In [48]:
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel

class AnswerWithJustification(BaseModel):
    '''An answer to the user's question along with justification for the answer.'''
    answer: str
    '''The answer to the user's question.'''
    justification: str
    '''Justification for the answer.'''


llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key, temperature=0)
structured_llm = llm.with_structured_output(AnswerWithJustification)

structured_llm.invoke("""What weighs more, a pound of bricks or a pound of feathers?""")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1534: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1547: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


AnswerWithJustification(answer='They weigh the same.', justification='Both a pound of bricks and a pound of feathers weigh one pound. The difference lies in the volume and density of the materials, not in their weight.')

### Example output parser

In [49]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
parser = CommaSeparatedListOutputParser()
parser.invoke("apples, oranges, bananas")

['apples', 'oranges', 'bananas']

### Using the Runnable Interface

In [51]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key=api_key)

model.batch(['Hi there!', 'Bye!'])

[AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEFxu2DOouLzLyQtgh33236tig3Ku', 'finish_reason': 'stop', 'logprobs': None}, id='run-e3d681d4-686c-459d-a788-5bc21677bdf7-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 AIMessage(content='Goodbye! Have a great day!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 10, 'total_tokens': 19, 'completion_tokens_de

### Imperative Composition


In [55]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain


template = ChatPromptTemplate.from_messages([
    ('system', """You are a helpful assistant."""),
    ('human','{question}'),])

model = ChatOpenAI(api_key=api_key)

@chain
def chatbot(values):
    prompt = template.invoke(values)
    return model.invoke(prompt)

chatbot.invoke({"question": "Which model providers offer Large Language Models?"})



AIMessage(content='Several companies offer Large Language Models through their cloud services or APIs. Some popular providers include:\n\n1. OpenAI with their GPT-3 model\n2. Google Cloud with the Cloud Natural Language API\n3. Microsoft Azure with Azure Cognitive Services Language APIs\n4. Amazon Web Services (AWS) with Amazon Comprehend\n5. IBM Watson with Watson Natural Language Understanding\n\nThese providers offer different features and pricing structures for using Large Language Models in various applications.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 25, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEG6XVMnWZ2u1rgUzdWiMrVQPdbM9', 'finish_reason': '

In [54]:
@chain
async def chatbot(values):
    prompt = await template.ainvoke(values)
    return await model.ainvoke(prompt)

await chatbot.ainvoke({"question": "Which model in providers offer Large Language Models?"})

AIMessage(content='Some well-known providers that offer Large Language Models are OpenAI with their GPT (Generative Pre-trained Transformer) models, Google with its BERT (Bidirectional Encoder Representations from Transformers) models, and Microsoft with its Turing models. These models are designed to handle a wide range of natural language processing tasks by utilizing large-scale pre-training on vast amounts of text data.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 26, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEG6SZSiLHm6nc8YYOF3kgxBVric6', 'finish_reason': 'stop', 'logprobs': None}, id='run-84ee4f8d-508e-41e7-a896-ee9d26891cf2-0', usage_metadata={'inp

### Declarative Composition

In [56]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])

model = ChatOpenAI(api_key=api_key)


chatbot = template | model

chatbot.invoke({"question": "Which model in providers offer Large Language Models?"})



AIMessage(content="Some providers that offer Large Language Models include OpenAI's GPT-3 model, Google's BERT (Bidirectional Encoder Representations from Transformers), and Facebook's RoBERTa (Robustly optimized BERT approach). These models are commonly used for natural language processing tasks such as text generation, translation, summarization, and more.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 26, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEG9BEbKqEvHvLDZsIR45F0j0Lqre', 'finish_reason': 'stop', 'logprobs': None}, id='run-101e68cf-61d7-4e20-b5db-a632d491ed6f-0', usage_metadata={'input_tokens': 26, 'output_tokens': 69, 'total_tokens': 95, 'input_toke